In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [6]:
def getAvailabilityAndRewards():
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            sign = i['current-epoch-performance']['current-epoch-signing-percent']
            perc = sign['current-epoch-signed']/sign['current-epoch-to-sign']
            if perc == 2/3:
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                reward[address] = reward_accumulated
    return reward

In [7]:
def getRewardsAndStatus(cutoff):
    reward = dict()
    status = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        if address in cutoff: 
            reward_accumulated = i['lifetime']['reward-accumulated']
            reward[address] = reward_accumulated
            epos_status = i['status']
            status[address] = epos_status
    return reward, status


In [8]:
def CN1_test():
    print("Test-CN1: Slow validator is never starved (should be able to sign blocks)")
    
    
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    while block < last_block:
        block = getBlockNumber()
    # get the validator's reward who just meets the 2/3 cut-off  
    cutoff_rewards = getAvailabilityAndRewards()
    if not cutoff_rewards:
        print('currently no one is at the availability cut-off: 2/3 sign percentage')
    else:
        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("new epoch first block reached", block, "will begin testing")
        next_rewards, status = getRewardsAndStatus()
        for k,v in next_rewards.items():
            reward_per_block = v - cutoff_rewards[k]
            if reward_per_block == 0 or status[k] == 'not eligible to be elected next epoch' \
            or status[k] == 'currently elected and not signing enough blocks to be eligible for election next epoch':
                print("Test-CN1: Fail")
                if reward_per_block == 0:
                    print("Slow validator doesn't get reward")
                if status[k] == 'not eligible to be elected next epoch' \
                or status[k] == 'currently elected and not signing enough blocks to be eligible for election next epoch':
                    print("Slow validator is no longer eligible")
            else:
                print("Test-CN1: Success")

    

In [9]:
CN1_test()

Test-CN1: Slow validator is never starved (should be able to sign blocks)
current and last block numbers 16556 16567
currently no one is at the availability cut-off: 2/3 sign percentage
